**Тестовое задание**

Приложение собирает и обрабатывает множество данных, в частности, одними из важных являются данные по сну. Записи могут приходить от разных источников - из одного или из нескольких. Их могут писать разные устройства (фитнес-трекер, клипса на подушке, умный матрас, смартфон, который слушает ваше дыхание) как небольшими кусочками, так и целыми записями про весь сон. Данные от разных источников могут пересекаться друг с другом. 
Наша задача - из всего этого многообразия получить однй запись, которая будет обозначать все время сна от начала до конца (то есть буквально сообщать, что пользователь уснул в 11 вечера и проснулся в 6 утра).

Разработчику была поставлена задача написать функцию, которая на вход принимает пришедшие записи сна и на выходе создает обобщающую запись с началом и окончанием сна. 

**Входные данные**

Приходящие данные имеют следующий формат:
1. Одна или несколько записей типа ASLEEP - время, когда пользователь спал.
2. Источник записи сна.

Если ASLEEP записей несколько, их необходимо объединять в случае, если между ними разница меньше одного часа. Если разница больше или равна часу, то это уже другой сон. То есть мы считаем что человек проснулся, пожил своей жизнью и дальше уже заснет второй раз. А если проснулся, перевернулся на другой бок и снова уснул - то это одна запись сна.

Например:
* 2020-09-13 23:00:00	- 2020-09-14 02:30:00	ASLEEP	Beddit
* 2020-09-14 03:00:00 - 2020-09-14 10:00:00	ASLEEP	Beddit

Объединятся в:
* 2020-09-13 23:00:00	- 2020-09-14 10:00:00	ASLEEP	Beddit


**Правила объединения данных**

Источников сна может быть несколько, необходимо выбрать один, исходя из представленных ниже правил. 
Проверка идет в порядке уменьшения приоритета, то есть сначала проверяем п.1, потом, если он не дал результатов, п.2 и так далее. Если после какого-то правила остался один источник, дальнейшей проверки не требуется:
1. Смотрим, какой источник был выбран в предыдущий день. Если в предыдущем дне не было такого источника, то продолжаем поиск на день раньше, и так до тех пор, пока не дойдем до даты регистрации пользователя, но не более, чем 14 дней назад от текущего дня. 
2. Выбираем источник, у которого количество пришедших записей за текущий сон больше, чем у остальных.
3. Выбираем источник, который ≠ iPhone (Apple Health и iPhone - разные источники).

Нужно учитывать, что записи могут пересекаться между собой.

Не учитывать запись, если за сутки пришло больше 17 часов сна от одного источника.

Ваша задача - протестировать работу этой функции, вводя данные, которые будут моделировать данные из реальной жизни.

**Как подготовить данные для тестирования функции:**

1. Скопировать из шаблона или создать в Google Sheets таблицу со следующими заголовками столбцов:
begin, end, value, source, где:
* begin - время начала записи сна
* end - время конца записи сна
* value - тип сна (всегда ASLEEP, других типов нет)
* source - источник данных по сну
2. В настройках доступа в Google Sheets необходимо изменить "Ограниченный доступ" на "Просматривать могут все в Интернете, у кого есть эта ссылка" и указать роль "Читатель". Пример (шаблон) документа с примером данных можно найти [здесь](https://docs.google.com/spreadsheets/d/14pVZX6dGLVaTPDquCNfqyyf8wvNLWkf61w8GiLLc8dc/edit#gid=0).
3. На одном листе можно делать больше одного кейса с записями фаз сна. То есть вы можете добавлять данные с разными датами.
4. Функция берет на выход данные и выбранную дату сна. Дата сна - дата, когда сон закончился, например:
* Сон с 2020-01-01 23:00 до 2020-01-02 08:00 - считается сном за 2020-01-02
* Сон с 2020-01-02 01:00 до 2020-01-02 08:00 - считается сном за 2020-01-02
* Сон с 2020-01-01 21:00 до 2020-01-01 23:00 - считается сном за 2020-01-01

**Как запустить функцию?**
В меню в верхнем левом углу нужно нажать Runtime -> Run all (Среда выполненеия -> Выполнить все) и немного подождать, пока загрузится страница. Внизу появится поле Link, туда нужно будет вставить ссылку на Google Sheets документ, в котором вы подготовили данные. Чтобы перезагрузить данные заново - тоже нужно будет нажать Runtime -> Run all, чтобы снова вставить ссылку и запустить функцию.
Потом вы сможете выбрать дату сна и запустить функцию. Результат будет выведен ниже.

**Каким должен быть результат?**

Результатом вашей работы должен стать отдельный документ с отчетом о тестировании этой функции, найденными багами и тест-кейсами, которые вы бы оставили для упрощения жизни другого тестировщика, который будет иметь дело с этой функцией.
Оформление и содержание - на ваше усмотрение. 
Ссылку на документ с результатом вашей работы нужно будет добавить в анкету по этому [адресу](https://welltory.typeform.com/to/SM5GBU9E).




In [1]:
#@title
import json
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from collections import namedtuple

from IPython.display import clear_output
from IPython.display import Javascript
from IPython.display import HTML

def get_full_info(df):
    max_interval_between_phases = 3600   
    key = ['IN_BED', 'SLEEP', 'ASLEEP', 'COMMON']

    df = df.sort_values(['begin', 'end'], ascending=[True, False])
    df['begin'] = df['begin'].apply(lambda x: pd.to_datetime(x, utc=False).tz_localize(None))    
    df['end'] = df['end'].apply(lambda x: pd.to_datetime(x, utc=False).tz_localize(None))
    df['diff'] = 0
    
    df['number_of_records'] = df.shape[0]
    
    df['prev_end'] = df['end'].shift(1)
    df['prev_end'] = df['prev_end'].fillna(df['end'].iloc[0])
    
    df['prev_start'] = df['begin'].shift(1)
    df['prev_start'] = df['prev_start'].fillna(df['begin'].iloc[0])
    
    df['diff'] = (df['begin'] - df['prev_end'])
    df['diff'] = df['diff'].apply(lambda x: x.total_seconds())

    df['big_interval'] = (df['diff'] > max_interval_between_phases)
    df['is_common_type_phase'] = df['value'].isin(key)
    
    df['max_end'] = np.NaN
    df['max_end'].iloc[0] = df['end'].iloc[0]  
    
    for i in range(1, df.shape[0]):
        df['max_end'].iloc[i] = max(df['max_end'].iloc[i-1], df['end'].iloc[i])

    df['prev_max_end'] = df['max_end'].shift(1)
    df['prev_max_end'].iloc[0] = df['end'].iloc[0]
    
    return df

def get_one_sleep_with_common_value(df):
    df = df.sort_values(['begin', 'end'])
    key = ['IN_BED', 'SLEEP','ASLEEP', 'COMMON']
    col = 'one_sleep_with_common_value'
    sleep_in_bed_begin, sleep_in_bed_end = None, None
    flags = []
    for i, row in df.iterrows():
        flag = None
        if row['value'] in key and not (sleep_in_bed_begin and sleep_in_bed_end):   
            sleep_in_bed_begin, sleep_in_bed_end = row['begin'], row['end']
            row['is_common_type_phase'] = True
            flag = True
        elif sleep_in_bed_begin and sleep_in_bed_end:
            flag = row['begin'] >= sleep_in_bed_begin and row['end'] <= sleep_in_bed_end
            if flag == False: 
                sleep_in_bed_begin, sleep_in_bed_end = row['begin'], row['end']
                row['is_common_type_phase'] = True
                flag = row['begin'] >= sleep_in_bed_begin and row['end'] <= sleep_in_bed_end
        else:
            flag = False

        flags.append({'flag_index': i, col: flag})
        
    return pd.merge(df, pd.DataFrame(flags), left_index=True, right_on='flag_index').drop('flag_index', axis=1)

def get_sleep_index(df):
    df['sleep_index'] = 0    
    
    df['to_next'] = df.apply(lambda x: True if ((x['big_interval'] and x['is_common_type_phase'] and x['one_sleep_with_common_value'] and x['max_end'] > x['prev_max_end']) or
                                               (x['big_interval'] and not x['is_common_type_phase'] and not x['one_sleep_with_common_value'] and x['max_end'] > x['prev_max_end']) or
                                               (x['big_interval'] and not x['is_common_type_phase'] and x['one_sleep_with_common_value'] and x['max_end'] > x['prev_max_end'])
                                               )
                             else False, axis=1)
    
    for i in range(1, df.shape[0]):
        df['sleep_index'].iloc[i] = df['sleep_index'].iloc[i-1] + df['to_next'].iloc[i]
        
    return df


def get_next_start_for_group(sleep_group):
    last_end_value = sleep_group['end'].iloc[-1]
    sleep_group['next_start'] = sleep_group['begin'].shift(-1) 
    sleep_group['next_start'] = sleep_group['next_start'].fillna(last_end_value)
    
    return sleep_group


def has_overall_record(sleep_group):
    sleep_group['is_overall_record'] = False
    sleep_group['has_overall_record'] = False
    sleep_group['overall_record_begin'] = np.NaN
    sleep_group['overall_record_end'] = np.NaN

    sleep_group = sleep_group.sort_values(['begin', 'end'], ascending=[True, False])
    if sleep_group['begin'].iloc[0] <= sleep_group['begin'].min() and sleep_group['end'].iloc[0] >= sleep_group['end'].max():
        sleep_group['has_overall_record'] = True
        sleep_group['is_overall_record'].iloc[0] = True
        sleep_group['overall_record_begin'] = sleep_group['begin'].iloc[0]
        sleep_group['overall_record_end'] = sleep_group['end'].iloc[0]
    else:
        sleep_group['has_overall_record'] = False
        
    return sleep_group

  
def is_good_data(sleep_group):
    sleep_group['good_data'] = sleep_group['end'] <= sleep_group['next_start']
    
    return sleep_group


def has_not_any_intersections(sleep_group):
    df = sleep_group[~sleep_group['is_overall_record']]
    sleep_group['has_not_any_intersections'] = all(df['good_data'])
    
    return sleep_group


def get_one_type_intersection(sleep_group):
    df = sleep_group[~sleep_group['is_overall_record']]

    if df['has_not_any_intersections'].all():
        sleep_group['has_one_type_intersection'] = False   
        return sleep_group

    has_one_type_intersection = []
    indexes_to_drop = set()
    for i in df.index[:-1]:
        next_index = i + 1
        if df['value'].loc[i] == df['value'].loc[next_index]:
            has_one_type_intersection.append(True) 
            sleep_group['begin'].loc[next_index] = min(sleep_group['begin'].loc[i], sleep_group['begin'].loc[next_index])
            sleep_group['end'].loc[next_index] = max(sleep_group['end'].loc[i], sleep_group['end'].loc[next_index])            

            indexes_to_drop.add(i)
        else:
            has_one_type_intersection.append(False)
    sleep_group['has_one_type_intersection'] = all(has_one_type_intersection)
    sleep_group = sleep_group.drop(index=indexes_to_drop)
    
    if sleep_group.shape[0] == 1:
        return sleep_group
    
    start_sleep_in_overall_record = sleep_group['begin'].iloc[0] <= sleep_group['begin'].iloc[1]
    end_sleep_in_overall_record = sleep_group['end'].iloc[0] >= sleep_group['end'].iloc[1]
    same_phase = sleep_group['value'].iloc[0] == sleep_group['value'].iloc[1]
    
    if same_phase and start_sleep_in_overall_record and end_sleep_in_overall_record:
        sleep_group['has_one_type_intersection'] = True
        sleep_group = sleep_group.drop(index=sleep_group.iloc[1].name)
        
    return sleep_group


def check_diff_between_rows(sleep_group):
    sleep_group['53_case_condition'] = False
    max_time_between_rows_for_53_case = 5400 # sec
    df = sleep_group.iloc[1:, :]

    if df.shape[0] > 1 and (df['diff'] > max_time_between_rows_for_53_case).any():
        sleep_group['53_case_condition'] = True
    if df.shape[0] == 1 and df['has_overall_record'].iloc[0] == True and (df['begin'].iloc[0] - df['prev_start'].iloc[0]).total_seconds() > max_time_between_rows_for_53_case:
        sleep_group['53_case_condition'] = True
    return sleep_group


def merge_same_phase_into_one_row(sleep_group):
    if sleep_group.shape[0] == 1:
        return sleep_group
    indexes_to_drop = set()
    for i in sleep_group[sleep_group['is_overall_record'] != True].index[:-1]:
        next_index = i + 1
        if (sleep_group['value'].loc[i] == sleep_group['value'].loc[next_index]) and ((sleep_group['begin'].loc[next_index] - sleep_group['end'].loc[i]).total_seconds() <= 1): 
            sleep_group['begin'].loc[next_index] = min(sleep_group['begin'].loc[i], sleep_group['begin'].loc[next_index])
            sleep_group['end'].loc[next_index] = max(sleep_group['end'].loc[i], sleep_group['end'].loc[next_index])            

            indexes_to_drop.add(i)
            
    return sleep_group.drop(index=indexes_to_drop)


def count_sleep_duration(sleep_group):
    sleep_group['duration_sec'] = sleep_group.apply(lambda x: (x['end'] - x['begin']).total_seconds(), axis=1) 
    
    return sleep_group


def get_sleep_group_data(sleep_group):
    awakes = ['AWAKE', 'FALLING_ASLEEP', 'WAKING_UP']
    sleep_group = sleep_group.sort_values('begin')
    begin_min = sleep_group['begin'].iloc[0]
    end_max = sleep_group['end'].iloc[-1]
    awakes_duration = sleep_group[sleep_group['value'].isin(awakes)]['duration_sec'].sum()
    d = {
        'begin' : [begin_min],
        'end'   : [end_max],
        'value' : ['GROUP'],
        'sleep_index' : [sleep_group['sleep_index'].iloc[0]],
        'source' : [sleep_group['source'].iloc[0]],
        'number_of_records' : [sleep_group['number_of_records'].iloc[0]],
        'duration_sec': [(end_max - begin_min - pd.Timedelta(seconds = awakes_duration)).total_seconds()],
        'artificial_value': [True],
        }  
    add_sleep_group = pd.DataFrame(data=d)
    sleep_group = sleep_group.append(add_sleep_group)

    return sleep_group


def clear_data(sleep_group):
    too_long_sleep = 61200 # sec
    if sleep_group.loc[sleep_group['value'] == 'GROUP']['duration_sec'].sum() > too_long_sleep:
        sleep_group = sleep_group.iloc[0:0]
        
    return sleep_group


def get_day(sleep_group):
    end_sleep_day = sleep_group['end'].iloc[-1].date()
    sleep_group['day'] = end_sleep_day
    
    return sleep_group


def drop_overall_record(sleep_group):
    if sleep_group.shape[0] <= 2:
        return sleep_group
    group = sleep_group[(sleep_group['value'] == 'GROUP')]
    overall_record = sleep_group[sleep_group['is_overall_record'] == True]
    if overall_record.empty:
        return sleep_group
    
    same_begin = group['begin'].iloc[0] == overall_record['begin'].iloc[0]
    same_end = group['end'].iloc[0] == overall_record['end'].iloc[0]
    if same_begin and same_end:
        sleep_group = sleep_group[sleep_group['is_overall_record'] != True]
    return sleep_group


def number_of_sleep_data_after_processing(sleep_group):
    sleep_group['numnber_of_rows_after_process'] = sleep_group.shape[0]
    return sleep_group


def main(df):
    # get case
    if df.empty:
        print('No data on source-layer')
        return df
    df = get_full_info(df)
    df = get_one_sleep_with_common_value(df)
    df = get_sleep_index(df)    
    df = df.groupby('sleep_index').apply(get_next_start_for_group).reset_index(drop=True)    
    df = df.groupby('sleep_index').apply(has_overall_record).reset_index(drop=True)    
    df = df.groupby('sleep_index').apply(is_good_data).reset_index(drop=True)  
    df = df.groupby('sleep_index').apply(has_not_any_intersections).reset_index(drop=True)    
    df = df.groupby('sleep_index').apply(get_one_type_intersection).reset_index(drop=True)

    # refresh next_start if change some rows
    df = df.groupby('sleep_index').apply(get_next_start_for_group).reset_index(drop=True)
    df = df.groupby('sleep_index').apply(is_good_data).reset_index(drop=True)    
    df = df.groupby('sleep_index').apply(check_diff_between_rows).reset_index(drop=True)    

    # refresh next_start if change some rows
    df = df.groupby('sleep_index').apply(get_next_start_for_group).reset_index(drop=True)
    df = df.groupby('sleep_index').apply(is_good_data).reset_index(drop=True)
    df = df.groupby('sleep_index').apply(merge_same_phase_into_one_row).reset_index(drop=True)   
    
    # refresh next_start if change some rows
    df = df.groupby('sleep_index').apply(get_next_start_for_group).reset_index(drop=True)
    df = df.groupby('sleep_index').apply(is_good_data).reset_index(drop=True)
    
    df = df.groupby('sleep_index').apply(count_sleep_duration)    
    df = df.groupby('sleep_index').apply(get_sleep_group_data).reset_index(drop=True)    
    df = df.groupby('sleep_index').apply(clear_data).reset_index(drop=True)    
    df = df.groupby('sleep_index').apply(get_day)
    
    #get source
    df = df.groupby('sleep_index').apply(drop_overall_record).reset_index(drop=True)  
    df = df.groupby('sleep_index').apply(number_of_sleep_data_after_processing).reset_index(drop=True)    
    return df


def check_yesterday_source(df, one_day_df):
    yesterday_sources = df.groupby('day')['source'].unique().iloc[0]
    one_day_df['same_source_yesterday'] = one_day_df['source'].isin(yesterday_sources)
    
    return one_day_df


def is_one_sleep(one_day_df):
    number_of_sleep_groups = one_day_df['value'].value_counts()['GROUP']
    if number_of_sleep_groups == 1:
        return one_day_df
    
    tmp = one_day_df[one_day_df['value'] == 'GROUP'].sort_values('begin')
    one_day_df['is_one_sleep'] = False
    tmp['sleep_group_intersec'] = np.NaN
    intersec_index = []
    for i in range (1, tmp.shape[0]):
        if tmp['begin'].iloc[i] < tmp['end'].iloc[i-1]:
            tmp['sleep_group_intersec'] = True
            one_day_df['is_one_sleep'] = True
            intersec_index.append(tmp['sleep_index'].iloc[i])
            intersec_index.append(tmp['sleep_index'].iloc[i-1])
        else:
             tmp['sleep_group_intersec'] = False
    if intersec_index:
        return one_day_df[one_day_df['sleep_index'].isin(intersec_index)]
    
    return one_day_df


def get_one_source(one_day_df):
    if one_day_df['source'].unique().shape[0] == 1:
        return one_day_df      
             
    tmp = one_day_df[one_day_df['source'] != 'iPhone']
    if tmp['source'].unique().shape[0] == 1:
        return tmp
    elif tmp['source'].unique().shape[0] > 1:
        one_day_df = tmp
        
    tmp = one_day_df[one_day_df['same_source_yesterday'] == True]
    if tmp['source'].unique().shape[0] == 1:
        return tmp
    elif tmp['source'].unique().shape[0] > 1:
        one_day_df = tmp
                      
    tmp = one_day_df[one_day_df['number_of_records'] == one_day_df['number_of_records'].max()]
    if tmp['source'].unique().shape[0] == 1:
        return tmp
    elif tmp['source'].unique().shape[0] > 1:
        one_day_df = tmp
        
    tmp = one_day_df[one_day_df['numnber_of_rows_after_process'] == one_day_df['numnber_of_rows_after_process'].max()]
    if tmp['source'].unique().shape[0] == 1:
        return tmp
    elif tmp['source'].unique().shape[0] > 1:
        one_day_df = tmp
        
    return one_day_df


def main_get_one_source(script_result, date): #date
    one_day_df = script_result[script_result['day'] == pd.to_datetime(date)] 
    if one_day_df.empty:
        return one_day_df
    if one_day_df['source'].unique().shape[0] == 1:
        return one_day_df
    
    one_day_df = one_day_df.apply(check_yesterday_source(script_result, one_day_df), axis=1)    
    one_day_df = is_one_sleep(one_day_df)   
    result = get_one_source(one_day_df)
    
    return result

Чтобы получить результат работы функции определения сна, необходимо ниже, в поле Link, вставить ссылку вашего Google sheets.




In [2]:
#@title
google_table_link = widgets.Text(
    value='',
    placeholder='Type here Google sheets link',
    description='Link:',
    disabled=False
)
google_table_link

Text(value='', description='Link:', placeholder='Type here Google sheets link')

Выбрать дату сна:

In [3]:
#@title
date_testing = widgets.DatePicker(
    description='Date',
    disabled=False)

display(date_testing)

DatePicker(value=None, description='Date')

И нажать на кнопку Get Result.

В итоге, появится таблица со следующими значениями:
* sleep_begin - время начала сна
* sleep_end - время, когда пользователь проснулся
* day - дата сна
* source - источник данных сна

In [4]:
#@title
button_get_sleep = widgets.Button(description='Get Result')
out_get_sleep = widgets.Output()

def on_button_clicked(_):  
    with out_get_sleep:
        clear_output()
        try:
          link = google_table_link.value
          key_1 = link.split('/')[5]

          df = pd.read_csv(  
              'https://docs.google.com/spreadsheets/d/' + key_1 +
              '/export?gid=0' + '&format=csv',
            index_col=0,
            ).reset_index()

          df = df.groupby(['source']).apply(main).reset_index(drop=True) 

          # date = df['day'].unique()[0]
          date = date_testing.value
          new_df = main_get_one_source(df, date)

          new_df = new_df[new_df['value'] == 'GROUP']

          new_df = new_df[[
              'begin',
              'end',
              'day',
              'source',  
          ]]

          new_df = new_df.rename(columns={
              'begin': 'sleep_begin',
              'end': 'sleep_end'
          })

          display(new_df.reset_index(drop=True))
        except IndexError as e:
          print('Insert Google Sheets Link')
              
button_get_sleep.on_click(on_button_clicked)
widgets.VBox([button_get_sleep, out_get_sleep])

* Если у вас появилось сообщение об ошибке HTTPError: HTTP Error 400: Bad Request, скорее всего вы забыли открыть доступ к вашему Google Sheets. Откройте доступ и выполните Runtime -> Run all (Среда выполненеия -> Выполнить все).
* Чтобы вернуть страницу с заданием к первоначальному виду, необходимо выполнить Runtime -> Run all (Среда выполненеия -> Выполнить все).
* Если не появилась кнопка Get Result, необходимо выполнить Runtime -> Run all (Среда выполненеия -> Выполнить все).

Если вы не смогли заставить все это работать и считаете, что это наша вина, напишите подробности на hr+qa@welltory.ru, мы проверим.
Ждем ваших результатов и надеемся, что это задание было для вас интересным.
Мы обязательно предоставим вам обратную связь вне зависимости от результата. 